In [101]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [102]:
import tensorflow as tf

n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [103]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")#[batch_size,n_inputs]
y = tf.placeholder(tf.int32, shape=(None), name="y")#[batch_size,1]

In [104]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])#[1,n_inputs]
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:#활성화 함수가 지정된 경우
            return activation(Z)#활성화 함수에 Z를 넣어 함수 값 반환
        else:
            return Z#아니면 Z를 반환

In [105]:
with tf.name_scope("dnn"):#이름 범위를 사용하여 정리
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)#[batch_size, 300]
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)#[batch_size, 100]
    logits = neuron_layer(hidden2, n_outputs, name="outputs")#[batch_size, 10]
    #활성화 함수에 들어가기 전

In [106]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    #[batch_size,1]
    loss = tf.reduce_mean(xentropy, name="loss")#[1]

In [107]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [108]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [109]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [110]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [111]:
n_epochs = 40
batch_size = 50

In [112]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [113]:
X_train = mnist.train.images
y_train = mnist.train.labels
X_valid = mnist.validation.images
y_valid = mnist.validation.labels
X_test = mnist.test.images
y_test = mnist.test.labels

In [114]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 배치 데이터 정확도: 0.9 검증 세트 정확도: 0.9148
1 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9358
2 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9448
3 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9514
4 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.956
5 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9566
6 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9614
7 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.963
8 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9648
9 배치 데이터 정확도: 0.96 검증 세트 정확도: 0.9662
10 배치 데이터 정확도: 0.92 검증 세트 정확도: 0.9684
11 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.969
12 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9662
13 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9704
14 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9712
15 배치 데이터 정확도: 0.94 검증 세트 정확도: 0.9736
16 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.973
17 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9742
18 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9746
19 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9746
20 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9752
21 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9756
22 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9762
23 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.9754
24 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.977
25 배치 데이터 정확도: 1.0 검증 세트 정확도: 0.9768
26 배치 데이터 정확도: 0.98 검증 세트 정확도: 0.978

In [115]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # 또는 save_path를 사용합니다
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [116]:
print("예측 클래스:", y_pred)
print("진짜 클래스:", y_test[:20])

예측 클래스: [7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
진짜 클래스: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [117]:
from tensorflow_graph_in_jupyter import show_graph

In [118]:
show_graph(tf.get_default_graph())